# Annotated Browsers for Journal-Specific Quotations in *Middlemarch*

In [1]:
import pandas as pd
import nltk
%matplotlib inline
import math
from ast import literal_eval
import numpy as np
import re
from matplotlib import pyplot as plt
#from colour import Color
from IPython.core.display import HTML
from matplotlib import cm
from matplotlib.colors import rgb2hex
plt.rcParams["figure.figsize"] = [16, 6]

In [2]:
df = pd.read_json('../data/e3.json')

In [3]:
df

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
0,"[[5809, 6218], [8751, 9046], [57013, 57100], [...","[[10456, 10865], [10994, 11737], [12404, 12491...",[Harriet Farwell Adams],[19840601],"[Language & Literature, Humanities]",10.2307/3044822,c6e6ce20-79c4-3c59-af91-b06c3208b37b,[ninecentfict],Nineteenth-Century Fiction,[eng],...,"<cite>Nineteenth-Century Fiction</cite>, Vol. ...","[Dorothea and ""Miss Brooke"" in Middlemarch]","[Sentiment, Fear, Martyrdom, Envy, Vocation, G...",fla,[39],1984,1980,1143,"[[[5809, 6218], 83], [[8751, 9046], 64], [[570...","[83, 64, 18, 29, 139, 27, 101, 21, 102, 140, 1..."
1,[],[],[HUGH WITEMEYER],[19910901],"[Language & Literature, Humanities]",10.2307/43470798,0d7eb58a-e4c1-326b-a195-012da1a4eb11,[georelioghlnews],"The George Eliot, George Henry Lewes Newsletter",[eng],...,"<cite>The George Eliot, George Henry Lewes New...",None,"[Lecture methods, Feminism, Pedagogy, Novelist...",brv,None,1991,1990,0,[],[]
2,[],[],"[Alison Cree, Louis J. Guillette, <suffix>Jr.<...",[19950601],"[Biological Sciences, Science and Mathematics,...",10.2307/1564553,f7384b7a-36be-3f0f-ac0b-b66455da0d36,[jherpetology],None,[eng],...,"<cite>Journal of Herpetology</cite>, Vol. 29, ...",[Biennial Reproduction with a Fourteen-Month P...,"[Animal vivipary, Parturition, Fat body, Skink...",fla,[29],1995,1990,0,[],[]
3,"[[1814, 1922], [1638943, 1639037], [1649154, 1...","[[23226, 23334], [25173, 25266], [27097, 27225]]",[Calvin Bedient],[19690401],"[Language & Literature, Humanities]",10.2307/3849222,ba20b1ad-b273-3608-bc9b-17b13f6d4e68,[hudsonreview],The Hudson Review,[eng],...,"<cite>The Hudson Review</cite>, Vol. 22, No. 1...",[Middlemarch: Touching Down],"[Immortality, Asceticism, Sentimentality, Mete...",fla,[22],1969,1960,71,"[[[1814, 1922], 20], [[1638943, 1639037], 24],...","[20, 24, 27]"
4,"[[397716, 398312], [1588815, 1589729], [158881...","[[20493, 21084], [22491, 23406], [22491, 23416]]",[Jane S. Smith],[19770701],"[Language & Literature, Humanities]",10.2307/40754482,ef12c01b-c42d-39b8-84dd-d00e6538143c,[texastudlitelang],Texas Studies in Literature and Language,[eng],...,<cite>Texas Studies in Literature and Language...,[The Reader as Part of the Fiction: Middlemarch],"[Novels, Flattery, Humor, Vanity, Meditation, ...",fla,[19],1977,1970,501,"[[[397716, 398312], 122], [[1588815, 1589729],...","[122, 189, 190]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6064,[],[],[Paul A. Brown],[19570401],"[Humanities, Language & Literature]",10.2307/2699152,2fa32cdd-0abb-37f8-ab1f-714e67c65582,[pmla],None,[eng],...,"<cite>PMLA</cite>, Vol. 72, No. 2 (Apr., 1957)...",[1956 Annual Bibliography],"[Music criticism, Eurythmy, Annotated bibliogr...",mis,[72],1957,1950,0,[],[]
6065,[],[],[Paul A. Brown],[19600501],"[Humanities, Language & Literature]",10.2307/2699365,c8f2a2ba-bf55-30e9-a744-bf9af751d955,[pmla],None,[eng],...,"<cite>PMLA</cite>, Vol. 75, No. 2 (May, 1960),...",[1959 Annual Bibliography],"[Onomastics, Music criticism, Periodicals, Ana...",mis,[75],1960,1960,0,[],[]
6066,[],[],[Paul A. Brown],[19590501],"[Humanities, Language & Literature]",10.2307/2699226,14b84b3e-49ed-3348-bf73-feb00bc75221,[pmla],None,[eng],...,"<cite>PMLA</cite>, Vol. 74, No. 2 (May, 1959),...",[1958 Annual Bibliography],"[Music criticism, Bibliographies, Onomastics, ...",mis,[74],1959,1950,0,[],[]
6067,[],[],None,"[20120101, 20120101]",[Language & Literature],10.3138/9781442662087.6,2416af8e-ce14-3993-ab70-42aa0b839b19,None,None,[eng],...,None,[Index to the Collected Works of Northrop Frye...,"[Myths, Satire, Catharsis, Mimesis, Masques, P...",mis,[30],2012,2010,0,[],[]


## *George Eliot - George Henry Lewes Studies* (*GE-GHLS*)

In [5]:
ge_ghls_df = df.loc[df['jcode'].str[0] == 'georelioghlstud']
ge_ghls_df

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
7,[],[],[K. K. Collins],[20151101],"[Language & Literature, Humanities]",10.5325/georelioghlstud.67.2.0234,6a9b9119-bfa6-3a6e-82cf-1dccbaefe55b,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[Review],"[Biography, Masculinity, Pastiche, Textbook re...",brv,[67],2015,2010,0,[],[]
14,"[[195357, 195597]]","[[27642, 27885]]","[ROSEMARY BRENNAN DAY, GRAHAM HANDLEY, DONALD ...",[19940901],"[Language & Literature, Humanities]",10.2307/42870840,72ab47e3-5bd7-3c8c-a03e-ee9281130e2d,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[MIDDLEMARCH ON TV--A SYMPOSIUM],"[Villains, Humor, Screenwriting, Pity, Soap op...",fla,None,1994,1990,45,"[[[195357, 195597], 45]]",[45]
34,"[[512101, 512375]]","[[9829, 10102]]",[LESLEY GORDON],[19950901],"[Language & Literature, Humanities]",10.2307/43595510,c7b950b2-0669-3b7e-b270-1f5935d046d2,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[GEORGE ELIOT AND PLUTARCH],"[Nightingales, Tyranny, Theater criticism, Nov...",fla,None,1995,1990,58,"[[[512101, 512375], 58]]",[58]
40,[],[],[JOHN R. PFEIFFER],[20070901],"[Language & Literature, Humanities]",10.2307/42827841,4107827d-116a-3e2e-a579-4d68d6549df3,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Novels, Literary characters, Essayists, Wheel...",brv,None,2007,2000,0,[],[]
46,"[[79888, 79968], [137185, 137297], [415296, 41...","[[2131, 2207], [3016, 3127], [7459, 7622], [77...",[CHESTER ST. H. MILLS],[19940901],"[Language & Literature, Humanities]",10.2307/42870836,bc692f3b-e2eb-3f26-9285-9203f5d6de48,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[ELIOT'S SPANISH CONNECTION: CASAUBON, THE AVA...","[Chivalry, Somatosensory perception, Endangere...",fla,None,1994,1990,281,"[[[79888, 79968], 19], [[137185, 137297], 30],...","[19, 30, 31, 97, 104]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4436,[],[],[DEBORAH GUTH],[19980901],"[Language & Literature, Humanities]",10.2307/42827672,27c44fbe-779e-3200-971e-2de9260955b2,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[GEORGE ELIOT AND SCHILLER: THE CASE OF ""THE M...","[Idealism, Pity, Heroism, Selfhood, Love, Nost...",fla,None,1998,1990,0,[],[]
4633,"[[404881, 405333], [441601, 441720]]","[[57303, 57753], [89214, 89333]]",[DAVID A REIBEL],[20131001],"[Language & Literature, Humanities]",10.2307/42827919,038e64cb-cfc8-3566-8868-4ac1e6d5464d,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[HIDDEN PARALLELS IN GEORGE ELIOT'S DANIEL DER...,"[Pity, Artistic talent, Orchestras, Cartoons, ...",fla,None,2013,2010,123,"[[[404881, 405333], 99], [[441601, 441720], 24]]","[99, 24]"
4997,[],[],[A. G. VAN DEN BROEK],[19930901],"[Language & Literature, Humanities]",10.2307/42827625,36986a59-df40-3cc0-85ca-7d835a60941e,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[SHAKESPEARE AT THE HEART OF GEORGE ELIOT'S EN...,"[Pity, Rebellion, Mottos, Literary epigraphs, ...",fla,None,1993,1990,0,[],[]
4998,[],[],[JENNIFER M. STOLPA],[20020901],"[Language & Literature, Humanities]",10.2307/42827747,813212aa-0b4e-30f5-8f52-26d2235ed0ce,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[DINAH AND THE DEBATE OVER VOCATION IN ""ADAM B...","[Preachers, Clergy, Feminism, Vocati

In [6]:
ge_ghls_df.to_json('../data/ge_ghls_data.json')

In [7]:
ge_ghls_1990s_df = ge_ghls_df[ge_ghls_df['Decade'] == 1990]
ge_ghls_1990s_df.head()

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
14,"[[195357, 195597]]","[[27642, 27885]]","[ROSEMARY BRENNAN DAY, GRAHAM HANDLEY, DONALD ...",[19940901],"[Language & Literature, Humanities]",10.2307/42870840,72ab47e3-5bd7-3c8c-a03e-ee9281130e2d,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[MIDDLEMARCH ON TV--A SYMPOSIUM],"[Villains, Humor, Screenwriting, Pity, Soap op...",fla,None,1994,1990,45,"[[[195357, 195597], 45]]",[45]
34,"[[512101, 512375]]","[[9829, 10102]]",[LESLEY GORDON],[19950901],"[Language & Literature, Humanities]",10.2307/43595510,c7b950b2-0669-3b7e-b270-1f5935d046d2,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[GEORGE ELIOT AND PLUTARCH],"[Nightingales, Tyranny, Theater criticism, Nov...",fla,None,1995,1990,58,"[[[512101, 512375], 58]]",[58]
46,"[[79888, 79968], [137185, 137297], [415296, 41...","[[2131, 2207], [3016, 3127], [7459, 7622], [77...",[CHESTER ST. H. MILLS],[19940901],"[Language & Literature, Humanities]",10.2307/42870836,bc692f3b-e2eb-3f26-9285-9203f5d6de48,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[ELIOT'S SPANISH CONNECTION: CASAUBON, THE AVA...","[Chivalry, Somatosensory perception, Endangere...",fla,None,1994,1990,281,"[[[79888, 79968], 19], [[137185, 137297], 30],...","[19, 30, 31, 97, 104]"
49,"[[234173, 234366], [281678, 281814], [539647, ...","[[9325, 9519], [13304, 13440], [13997, 14107],...",[KEITH A. WADDLE],[19950901],"[Language & Literature, Humanities]",10.2307/43595509,8900103f-bb17-38f8-94aa-ad77ca8ca07f,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[MARY GARTH, THE WOLLSTONECRAFTIAN FEMINIST OF...","[Pity, Women, Passion, Humor, Compassion, Hone...",fla,None,1995,1990,179,"[[[234173, 234366], 44], [[281678, 281814], 30...","[44, 30, 27, 52, 26]"
75,[],[],[ELIZABETH WINSTON],[19950901],"[Language & Literature, Humanities]",10.2307/43595523,7acdda65-ea4c-3c1c-aa20-1f0e59773b19,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Novels, Serial publications, Adumbration, Gos...",brv,None,1995,1990,0,[],[]


In [8]:
ge_ghls_1990s_df.to_json('../data/ge_ghls_1990s_data.json')

In [9]:
ge_ghls_2010s_df = ge_ghls_df[ge_ghls_df['Decade'] == 2010]
ge_ghls_2010s_df.head()

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
7,[],[],[K. K. Collins],[20151101],"[Language & Literature, Humanities]",10.5325/georelioghlstud.67.2.0234,6a9b9119-bfa6-3a6e-82cf-1dccbaefe55b,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[Review],"[Biography, Masculinity, Pastiche, Textbook re...",brv,[67],2015,2010,0,[],[]
48,[],[],[Claudia Stevens],[20151101],"[Language & Literature, Humanities]",10.5325/georelioghlstud.67.2.0227,6dfa8330-d590-35d4-8e42-5686f97cf016,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[A New Opera: <em>Middlemarch in Spring</em>],"[Opera, Solos, Orchestras, Uncles, Ragpickers,...",fla,[67],2015,2010,0,[],[]
97,"[[490324, 490390], [513891, 514153], [514696, ...","[[8092, 8157], [9348, 9609], [9852, 10238], [1...",[NICOLE M. COONRADT],[20120901],"[Language & Literature, Humanities]",10.2307/42827900,8a9789c8-ad8c-387a-88ef-37aab78b949f,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[WRITING MARY GARTH: LOCATING MIDDLE GROUND AM...,"[Women, Omniscient narrators, Ugliness, Pity, ...",fla,None,2012,2010,425,"[[[490324, 490390], 13], [[513891, 514153], 52...","[13, 52, 80, 41, 49, 129, 61]"
119,[],[],[MICAEL M. CLARKE],[20131001],"[Language & Literature, Humanities]",10.2307/42827927,4a3436b7-92f0-38bf-82b4-750744974234,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Novels, Crime fiction, Literary characters, M...",brv,None,2013,2010,0,[],[]
146,[],[],"[J. HILLIS MILLER, DONALD HAWES, JOHN R. PFEIF...",[20100901],"[Language & Literature, Humanities]",10.2307/42827878,9439fdb7-36c1-36d4-8b45-c480d2b5c4c8,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Literary criticism, Humor, Irony, Humanism, S...",brv,None,2010,2010,0,[],[]


In [10]:
ge_ghls_2010s_df.to_json('../data/ge_ghls_2010s_data.json')

## *ELH*

In [11]:
elh_df = df.loc[df['jcode'].str[0] == 'elh']
elh_df

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
19,"[[415187, 415972], [1691326, 1691647]]","[[8469, 9251], [18464, 18785]]",[Janet K. Gezari],[19780401],"[Language & Literature, Humanities]",10.2307/2872453,bdec7777-4547-380f-8567-cb4dbcaa9388,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 45, No. 1 (Spring, 1978...",[The Metaphorical Imagination of George Eliot],"[Novels, Sympathy, Travel, Dialectic, Allegory...",fla,[45],1978,1970,228,"[[[415187, 415972], 158], [[1691326, 1691647],...","[158, 70]"
26,"[[698796, 699017], [1368267, 1368353], [169094...","[[15380, 15597], [29093, 29180], [40262, 40962]]",[Brian Swann],[19720601],"[Language & Literature, Humanities]",10.2307/2872247,af76839c-e17f-336c-a45b-7e2a150cf963,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 39, No. 2 (Jun., 1972),...",[Middlemarch: Realism and Symbolic Form],"[Synecdoche, Irony, Pity, Symbolism, Chivalry,...",fla,[39],1972,1970,231,"[[[698796, 699017], 57], [[1368267, 1368353], ...","[57, 15, 159]"
28,"[[18311, 18654], [36712, 36879], [39525, 39677...","[[9299, 9639], [13482, 13644], [22259, 22414],...",[Sophia Andres],[19881201],"[Language & Literature, Humanities]",10.2307/2873139,682a0b66-d9b2-305c-99dd-22cd29b16067,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 55, No. 4 (Winter, 1988...",[The Germ and the Picture in Middlemarch],"[Labyrinths, Preludes, Diastole, Harps, Curtai...",fla,[55],1988,1980,516,"[[[18311, 18654], 79], [[36712, 36879], 36], [...","[79, 36, 26, 46, 20, 15, 17, 29, 34, 15, 68, 1..."
72,"[[291679, 291940], [297602, 297690], [307281, ...","[[1777, 2037], [4485, 4590], [5770, 5987], [61...",[Jeremy Tambling],[19901201],"[Language & Literature, Humanities]",10.2307/2873091,4754a86e-dbdb-3cf2-aa1a-0179494fcc8c,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 57, No. 4 (Winter, 1990...","[Middlemarch, Realism and the Birth of the Cli...","[Metonymy, Constructive empiricism, Positivism...",fla,[57],1990,1990,466,"[[[291679, 291940], 52], [[297602, 297690], 17...","[52, 17, 44, 24, 53, 49, 79, 83, 37, 28]"
104,"[[176340, 176439], [312053, 312429], [739568, ...","[[8798, 8898], [15587, 15991], [18709, 18985],...",[MARK ALLISON],[20111001],"[Humanities, Language & Literature]",10.2307/41236564,16ed41f1-35c6-3da3-86e1-b4ef8e34ece2,[elh],None,[eng],...,"<cite>ELH</cite>, Vol. 78, No. 3 (FALL 2011), ...","[UTOPIAN SOCIALISM, WOMEN'S EMANCIPATION, AND ...","[Utopianism, Womens suffrage movements, Pity, ...",fla,[78],2011,2010,187,"[[[176340, 176439], 22], [[312053, 312429], 80...","[22, 80, 54, 31]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,[],[],[John P. Farrell],[19890401],"[Language & Literature, Humanities]",10.2307/2873128,b12be317-31aa-3111-a0ff-27e54d33a1f0,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 56, No. 1 (Spring, 1989...",[Reading the Text of Community in Wuthering He...,"[Dialogism, Narratology, Storytelling, Hierogl...",fla,[56],1989,1980,0,[],[]
5373,[],[],[Amit Yahav-Brown],[20060101],"[Language & Literature, Humanities]",10.2307/30030039,f8de8475-4690-3c3b-ad44-679c28d88ea2,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 73, No. 4 (Winter, 2006...",[Reasonableness and Domestic Fiction],"[Reason, Pluralist school, Letter writing, Par...",fla,[73],2006,2000,0,[],[]
5376,[],[],[William Deresiewicz],[19970701],"[Language & Literature, Humanities]",10.2307/30030146,4321709c-1c11-337b-a7d1-7d45878a15c0,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 64, No. 2 (Summer, 1997...","[Community and Cognition in ""Pride and Prejudi...","[Irony, Humor, Intimacy, Gossip, Flattery, Com...",fla,[64],1997,1990,0,[],[]
5383,[],[],[Rebecca F. Stern],[19980701],"[Language & Literature, Humanities]",10.2307/30030186,0f0a5ad2-54e6-3dc7-8bd7-b8a2c89edbef,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 65, No. 2 (Summer, 1998...",[Moving Parts and Speaking Parts: Situating Vi...,"[Spectacle, S

In [12]:
elh_df.to_json('../data/elh_data.json')

In [13]:
elh_df['Decade'].value_counts()

2000    21
1980    15
1990    14
1970    13
2010    12
1960     6
1950     2
1940     1
1930     1
Name: Decade, dtype: int64

In [14]:
elh_1960s_df = elh_df[elh_df['Decade'] == 1960]
elh_1970s_df = elh_df[elh_df['Decade'] == 1970]
elh_1990s_df = elh_df[elh_df['Decade'] == 1990]
elh_2010s_df = elh_df[elh_df['Decade'] == 2010]

In [15]:
elh_1960s_df.to_json('../data/elh_1960s_data.json')
elh_1970s_df.to_json('../data/elh_1970s_data.json')
elh_1990s_df.to_json('../data/elh_1990s_data.json')
elh_2010s_df.to_json('../data/elh_2010s_data.json')

## Browsers


In [16]:
## Change the JSON file for different dataframes
browser_df = pd.read_json('../data/elh_2010s_data.json')

In [17]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [18]:
mm = Text('../middlemarch.txt')

In [19]:
# Get the size of the text. 
textALength = 1793449

# I don't know why, but without the offset the novel ends too soon,
# with "unvisited tomb." This fixes it. 
offset = 2
textALength += offset

# Make an empty array the size of the text. 
tally = np.zeros(textALength, dtype=np.int)

<ipython-input-19-c9b92fc893f9>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tally = np.zeros(textALength, dtype=np.int)


In [20]:
len(tally)

1793451

In [21]:
# Read the locations from the CSV file, and literally evaluate them into lists. 
locations = browser_df['Locations in A']
#locations = locations.apply(literal_eval)

In [22]:
#type(locations[0])

In [23]:
# Tally up every time a letter in the novel is quoted. 
for article in locations: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [24]:
mm.text[1793218:1793418]

'growing good of the world is partly\ndependent on unhistoric acts; and that things are not so ill with you\nand me as they might have been, is half owing to the number who lived\nfaithfully a hidden life'

In [25]:
# Make a color list in hex for all the values in the tally. 
# Let's hope there aren't too many. 
normalized = [int(round((i/tally.max() * 255))) for i in range(tally.max())]
colorList = [rgb2hex(x) for x in cm.gnuplot(normalized)]

In [26]:
# Create a CSS Stylesheet for each color value in the map. 
colorCSS = ""
for i, color in zip(range(0, tally.max()+1), colorList): 
    colorCSS += ".c-%s { color: %s; }\n" % (i, color)

In [27]:
n = 50

checkpoints = np.linspace(0, textALength, n).round()
checkpoints = [int(point) for point in checkpoints]

In [28]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, mm.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    if i in checkpoints: 
        out += '<a name="b-%s"></a>' % checkpoints.index(i)
    out += char
    previousVal = val

In [29]:
len(browser_df)

12

In [30]:
# Make a list of valid decades. 
decades = np.arange(1930, 2020, 10)

# Make a dictionary of decades. 
# Values are a list of locations.  
decadeDict = {}
for i, row in df.iterrows():
    decade = row['Decade']
    locations = row['Locations in A']
    if decade not in decadeDict: 
        decadeDict[decade] = locations
    else: 
        decadeDict[decade] += locations 

In [31]:
# Grab the beginnings of quotes. 
decadeStarts = {decade: [item[0] for item in loc] for decade, loc in decadeDict.items()}

decadesBinned = {decade: 
                 np.histogram(locations, bins=n, range=(0, textALength))[0]
                 for decade, locations in decadeStarts.items() if decade in decades}

decadesDF = pd.DataFrame(decadesBinned).T

In [32]:
# Normalize the totals for each section. 
normalizedBlocks = decadesDF.sum() / decadesDF.sum().max()

# Now use the scale that we're already using for the CSS. 
normalizedBlocks = round(normalizedBlocks * tally.max())

In [33]:
blockHTML = '<section id="blocks">'
for i, block in enumerate(normalizedBlocks): 
    blockHTML += '<div class="block b-%s"><a class="block" href="#b-%s">%s</a></div>' % (int(block), i, i)
blockHTML = blockHTML + "</section>"

In [34]:
blockCSS = """
#blocks {  }
.block, .block { 
  width: 30px; 
  height: 30px; 
  display: inline-block;
  color: white; 
  text-align: center;
  text-decoration: none;
  margin-top: 3px; 
}
"""

for i, color in zip(range(0, tally.max()+1), colorList): 
    blockCSS += '.b-%s { background-color: %s; }\n' % (i, color)
colorCSS += blockCSS

In [35]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  pre { 
      font-family: Raleway, serif; 
      font-size: 18px; 
    }
  main { 
      width: 40em; 
      margin: 2em auto; 
  }
  %s
  </style>
</head>
<body>%s<main><pre>%s</pre></main></body></html>
""" % (colorCSS, blockHTML, out)

In [ ]:
with open('../annotated_elh_2010s_data.html', 'w') as f: 
    f.write(html)
    f.close()